## Set Up ##

In [1]:
import os
import shutil
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
import openpyxl
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
import pandas as pd
import xlwings as xw
import numpy as np
start_master = time.time()

In [2]:
download_path = input("Enter path to folder where everything is downloaded: ")
#C:\Users\stephalin\Downloads
raw_Staff = os.path.join(download_path, input("Enter Staffing source file name: ") + ".xlsx")
raw_EP = os.path.join(download_path, input("Enter Execution Pivot source file name: ") + ".xlsx")
src_folder = os.path.join(download_path, input("Enter source folder name: "))

#raw_Cloud = os.path.join(download_path, input("Enter ESC Cloud source file name: ") + ".csv")

raw_CostPool = os.path.join(download_path,"TBM ESC Cost Pool.xlsx")
raw_Sol = os.path.join(download_path,"TBM ESC Solutions.xlsx")
raw_T = os.path.join(download_path,"TBM ESC Tower.xlsx")


Enter path to folder where everything is downloaded:  C:\Users\stephalin\Downloads
Enter Staffing source file name:  RMS Staffing Projections & Actuals_20250806
Enter Execution Pivot source file name:  RMS Execution Pivot Report_20250806
Enter source folder name:  FY25 June - Q3


### Copying and renaming source folder ###

In [3]:
today = datetime.today()

#to test on last month's data, uncomment
#today = today - relativedelta(months=1)

print("Today is:", today)
first_of_month = today.replace(day=1)
prior_month_date = first_of_month - timedelta(days=1)
print("We will create data inputs for", prior_month_date.strftime("%B"), prior_month_date.year, "dashboard")

Today is: 2025-08-27 17:12:44.589191
We will create data inputs for July 2025 dashboard


In [4]:
#src_folder = r'C:\Users\stephalin\Downloads\FY25 June - Q3'
#dst_folder = r'C:\Users\stephalin\Downloads\FY25 July'
first_of_month = today.replace(day=1)
prior_month_date = first_of_month - timedelta(days=1)
year_str = str(prior_month_date.year)[-2:]
month_str = prior_month_date.strftime("%B")
quarter_map = {3: "Q1", 6: "Q2", 9: "Q3", 12: "Q4"}
quarter_suffix = ""
if prior_month_date.month in quarter_map:
    quarter_suffix = f" - {quarter_map[prior_month_date.month]}"

parent_folder = os.path.dirname(src_folder)
dst_folder_name = f"FY{year_str} {month_str}{quarter_suffix}"
dst_folder = os.path.join(parent_folder, dst_folder_name)

date_str = datetime.now().strftime('%Y%m%d')

os.makedirs(dst_folder, exist_ok=True)
for root, dirs, files in os.walk(src_folder):
    for filename in files:
        ext = os.path.splitext(filename)[1].lower()
        src_path = os.path.join(root, filename)
        if ext == ".xlsm":
            new_name = f"Model Enhancements_{date_str}{ext}"
        elif "Golden PowerBI Source File" in filename and ext == ".xlsx":
            new_name = f"Golden PowerBI Source File_{date_str}{ext}"
        elif ext == ".pbix":
            new_name = f"FY2025 OCIO TBM Analytics{ext}"
        else:
            continue
        shutil.copy2(src_path, os.path.join(dst_folder, new_name))

print("Copying contents of " + src_folder + " into " + dst_folder)
raw_data_src = os.path.join(src_folder, "Raw Data")
raw_data_dst = os.path.join(dst_folder, "Raw Data")
if os.path.exists(raw_data_src):
    shutil.copytree(raw_data_src, raw_data_dst, dirs_exist_ok=True)

for filename in os.listdir(raw_data_dst):
    file_path = os.path.join(raw_data_dst, filename)
    if os.path.isfile(file_path):
        base, ext = os.path.splitext(filename)
        base = base.split("_")[0]
        new_name = f"{base}_{date_str}{ext}"
        new_path = os.path.join(raw_data_dst, new_name)
        os.rename(file_path, new_path)
print("All contents have been copied and renamed in "+ dst_folder)


Copying contents of C:\Users\stephalin\Downloads\FY25 June - Q3 into C:\Users\stephalin\Downloads\FY25 July
All contents have been copied and renamed in C:\Users\stephalin\Downloads\FY25 July


## Refreshing data inputs ##

### Execution Pivot ###

In [5]:
start1 = time.time()
dst_EP = raw_data_dst+"\RMS Execution Pivot Report_"+date_str+".xlsx"
print("Copying contents of "+raw_EP+" into "+dst_EP)
print(" ")

Copying contents of C:\Users\stephalin\Downloads\RMS Execution Pivot Report_20250806.xlsx into C:\Users\stephalin\Downloads\FY25 July\Raw Data\RMS Execution Pivot Report_20250827.xlsx
 


<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\1570110182.py:2: SyntaxWarning: invalid escape sequence '\R'
  dst_EP = raw_data_dst+"\RMS Execution Pivot Report_"+date_str+".xlsx"


In [6]:
# Read raw data
raw_df = pd.read_excel(raw_EP, sheet_name="Sheet1")

# Cleaned data
df = raw_df.rename(columns={
    "UFMS Vendor Recipient": "Vendor Recipient",
    "Vendor Name": "Vendor POC Name",
    "Vendor Email": "Vendor POC Email"
})
delete_cols = [
    "Projection Note",
    "Line Item Mechanism Target Award Date",
    "RAL",
    "Late Waiver Number",
    "Late Waiver Deadline",
    "Unreconciled Commitments",
    "Unreconciled Obligations"
]
df = df.drop(columns=delete_cols, errors="ignore")

with xw.App(visible=False) as app:
    wb = app.books.open(dst_EP)
    # Write cleaned data to "Sheet1"
    try:
        wb.sheets["Sheet1"].delete()
    except Exception:
        pass
    clean_sht = wb.sheets.add("Sheet1", before=wb.sheets[0])
    clean_sht.range("A1").options(index=False).value = df

    nrows, ncols = df.shape
    # +1 for header row
    table_range = clean_sht.range((1, 1), (nrows + 1, ncols)).api
    clean_table = clean_sht.api.ListObjects.Add(1, table_range, None, 1)
    clean_table.Name = "Table13"
    clean_table.TableStyle = "TableStyleMedium2"

    wb.save()
    wb.close()


#copy into raw data tab

In [7]:
end1 = time.time()
print(f"Elapsed time: {end1 - start1} seconds for EP")

Elapsed time: 18.790987253189087 seconds for EP


### Staffing Report ###

In [8]:
start2 = time.time()
dst_Staff = raw_data_dst+"\RMS Staffing Projections & Actuals_"+date_str+".xlsx"
print("Copying contents of "+raw_Staff+" into "+dst_Staff)

Copying contents of C:\Users\stephalin\Downloads\RMS Staffing Projections & Actuals_20250806.xlsx into C:\Users\stephalin\Downloads\FY25 July\Raw Data\RMS Staffing Projections & Actuals_20250827.xlsx


<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\4235163783.py:2: SyntaxWarning: invalid escape sequence '\R'
  dst_Staff = raw_data_dst+"\RMS Staffing Projections & Actuals_"+date_str+".xlsx"


In [9]:
#Staffing report

df = pd.read_excel(raw_Staff, sheet_name="Sheet1")
wcf_map = pd.read_excel(dst_Staff, sheet_name="WCF Mapping")

# Remove 'AFPS' from column headers
df.columns = [col.replace("AFPS", "").strip() for col in df.columns]

# Insert Full Name column
def full_name_formula(row):
    if pd.notnull(row.iloc[0]) and "Budget Staff Adjustment" in str(row.iloc[0]):
        return ""
    if (str(row.get("Last Name", "")) + str(row.get("First Name", ""))).strip() == "":
        return "Vacant"
    return f"{row.get('Last Name','')}, {row.get('First Name','')}".strip(", ")
record_id_idx = df.columns.get_loc("Record ID")
df.insert(record_id_idx, "Full Name", df.apply(full_name_formula, axis=1))

# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("Total Amount") + 1

# Insert in reverse order
df.insert(total_amt_idx, "OCIO Assigned Office", "")
df.insert(total_amt_idx, "OCIO Funding Office", "")
df.insert(total_amt_idx, "Service Line", "")

In [10]:
# Write DataFrame to "Import Data" sheet
with pd.ExcelWriter(dst_Staff, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="Import Data", index=False)

wb = openpyxl.load_workbook(dst_Staff)
ws = wb["Import Data"]

header = [cell.value for cell in ws[1]]
wcf_indicator_col = header.index("Payroll Dist. WCF Indicator") + 1
employee_admin_code_col = header.index("Employee Admin Code") + 1
service_line_col = header.index("Service Line") + 1
funding_office_col = header.index("OCIO Funding Office") + 1
assigned_office_col = header.index("OCIO Assigned Office") + 1

for row in range(2, ws.max_row + 1):
    wcf_cell = f"{openpyxl.utils.get_column_letter(wcf_indicator_col)}{row}"
    emp_code_cell = f"{openpyxl.utils.get_column_letter(employee_admin_code_col)}{row}"

    ws.cell(row=row, column=service_line_col).value = (
        f'=VLOOKUP({wcf_cell},\'WCF Mapping\'!A:L,4,FALSE)'
    )
    ws.cell(row=row, column=funding_office_col).value = (
        f'=IFERROR(VLOOKUP(CONCATENATE("H",LEFT({emp_code_cell},FIND(" ",{emp_code_cell},1)-1)),' +
        "'OCIO Admin Codes'!A:I,9,FALSE),\"\")"
    )
    ws.cell(row=row, column=assigned_office_col).value = (
        f'=IFERROR(VLOOKUP(CONCATENATE("H",LEFT({emp_code_cell},FIND(" ",{emp_code_cell},1)-1)),' +
        "'OCIO Admin Codes'!A:I,9,FALSE),\"\")"
    )


In [11]:
max_row = ws.max_row
max_col = ws.max_column
end_col = openpyxl.utils.get_column_letter(max_col)
table_range = f"A1:{end_col}{max_row}"
table = Table(displayName="Table1", ref=table_range)
style = TableStyleInfo(name="TableStyleMedium2", showRowStripes=True)
table.tableStyleInfo = style
ws.add_table(table)
wb.save(dst_Staff)

In [12]:
end2 = time.time()
print(f"Elapsed time: {end2 - start2} seconds for Staffing")

Elapsed time: 22.701268911361694 seconds for Staffing


### TBM ESC ###

In [13]:
start3 = time.time()
dst_CostPool = raw_data_dst+"\TBM ESC Cost Pool_"+date_str+".xlsx"
print("Copying contents of "+raw_CostPool+" into "+dst_CostPool)

Copying contents of C:\Users\stephalin\Downloads\TBM ESC Cost Pool.xlsx into C:\Users\stephalin\Downloads\FY25 July\Raw Data\TBM ESC Cost Pool_20250827.xlsx


<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\2298528951.py:2: SyntaxWarning: invalid escape sequence '\T'
  dst_CostPool = raw_data_dst+"\TBM ESC Cost Pool_"+date_str+".xlsx"


In [14]:
df = pd.read_excel(raw_CostPool, sheet_name="TBM ESC Cost Pool", header=1)
df = df.iloc[1:].reset_index(drop=True)

# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("CP Amount") + 1

# Insert in reverse order
df.insert(total_amt_idx, "Service Line", "")
df.insert(total_amt_idx, "Office", "")
df.insert(total_amt_idx, "In OCIO?", "")

with pd.ExcelWriter(dst_CostPool, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="TBM ESC Cost Pool", index=False)

wb = openpyxl.load_workbook(dst_CostPool)
ws = wb["TBM ESC Cost Pool"]


In [15]:
header = [cell.value for cell in ws[1]]
admin_code_col = header.index("Admin Code") + 1
investment_id_col = header.index("Investment ID") + 1
in_ocio_col = header.index("In OCIO?") + 1
office_col = header.index("Office") + 1
service_line_col = header.index("Service Line") + 1

for row in range(2, ws.max_row + 1):
    p_cell = f"{openpyxl.utils.get_column_letter(admin_code_col)}{row}"
    j_cell = f"{openpyxl.utils.get_column_letter(investment_id_col)}{row}"
    ws.cell(row=row, column=in_ocio_col).value = f'=ISNUMBER(SEARCH("HCAJR",{p_cell}))'
    ws.cell(row=row, column=office_col).value = (f'=IF(ISNA(VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE)),"",VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE))')    
    ws.cell(row=row, column=service_line_col).value = (f'=IF(ISNA(VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE)),"Other - Non OCIO",VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE))')   

wb.save(dst_CostPool)

In [16]:
dst_Sol = raw_data_dst+"\TBM ESC Solutions_"+date_str+".xlsx"
print("Copying contents of "+raw_Sol+" into "+dst_Sol)

df = pd.read_excel(raw_Sol, sheet_name="TBM ESC Solutions", header=1)
df = df.iloc[1:].reset_index(drop=True)

# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("Amount") + 1

# Insert in reverse order
df.insert(total_amt_idx, "Service Line", "")
df.insert(total_amt_idx, "Office", "")
df.insert(total_amt_idx, "In OCIO?", "")

with pd.ExcelWriter(dst_Sol, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="TBM ESC Solutions", index=False)

wb = openpyxl.load_workbook(dst_Sol)
ws = wb["TBM ESC Solutions"]


header = [cell.value for cell in ws[1]]
admin_code_col = header.index("Admin Code") + 1
investment_id_col = header.index("Investment ID") + 1
in_ocio_col = header.index("In OCIO?") + 1
office_col = header.index("Office") + 1
service_line_col = header.index("Service Line") + 1

for row in range(2, ws.max_row + 1):
    p_cell = f"{openpyxl.utils.get_column_letter(admin_code_col)}{row}"
    j_cell = f"{openpyxl.utils.get_column_letter(investment_id_col)}{row}"
    ws.cell(row=row, column=in_ocio_col).value = f'=ISNUMBER(SEARCH("HCAJR",{p_cell}))'
    ws.cell(row=row, column=office_col).value = (f'=IF(ISNA(VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE)),"",VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE))')    
    ws.cell(row=row, column=service_line_col).value = (f'=IF(ISNA(VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE)),"Other - Non OCIO",VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE))')   

wb.save(dst_Sol)

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\1060953386.py:1: SyntaxWarning: invalid escape sequence '\T'
  dst_Sol = raw_data_dst+"\TBM ESC Solutions_"+date_str+".xlsx"


Copying contents of C:\Users\stephalin\Downloads\TBM ESC Solutions.xlsx into C:\Users\stephalin\Downloads\FY25 July\Raw Data\TBM ESC Solutions_20250827.xlsx


In [17]:
dst_T = raw_data_dst+"\TBM ESC Tower_"+date_str+".xlsx"
print("Copying contents of "+raw_T+" into "+dst_T)

df = pd.read_excel(raw_T, sheet_name="TBM ESC Tower", header=1)
df = df.iloc[1:].reset_index(drop=True)

# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("Amount") + 1

# Insert in reverse order
df.insert(total_amt_idx, "Service Line", "")
df.insert(total_amt_idx, "Office", "")
df.insert(total_amt_idx, "In OCIO?", "")

with pd.ExcelWriter(dst_T, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="TBM ESC Tower", index=False)

wb = openpyxl.load_workbook(dst_T)
ws = wb["TBM ESC Tower"]

header = [cell.value for cell in ws[1]]
admin_code_col = header.index("Admin Code") + 1
investment_id_col = header.index("Investment ID") + 1
in_ocio_col = header.index("In OCIO?") + 1
office_col = header.index("Office") + 1
service_line_col = header.index("Service Line") + 1

for row in range(2, ws.max_row + 1):
    p_cell = f"{openpyxl.utils.get_column_letter(admin_code_col)}{row}"
    j_cell = f"{openpyxl.utils.get_column_letter(investment_id_col)}{row}"
    ws.cell(row=row, column=in_ocio_col).value = f'=ISNUMBER(SEARCH("HCAJR",{p_cell}))'
    ws.cell(row=row, column=office_col).value = (f'=IF(ISNA(VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE)),"",VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE))')    
    ws.cell(row=row, column=service_line_col).value = (f'=IF(ISNA(VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE)),"Other - Non OCIO",VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE))')   

wb.save(dst_T)


<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\2680103817.py:1: SyntaxWarning: invalid escape sequence '\T'
  dst_T = raw_data_dst+"\TBM ESC Tower_"+date_str+".xlsx"


Copying contents of C:\Users\stephalin\Downloads\TBM ESC Tower.xlsx into C:\Users\stephalin\Downloads\FY25 July\Raw Data\TBM ESC Tower_20250827.xlsx


In [18]:
end3 = time.time()
print(f"Elapsed time: {end3 - start3} seconds for TBM ESC files")

Elapsed time: 83.49015378952026 seconds for TBM ESC files


In [19]:
print(f"Elapsed time: {end3 - start_master} seconds for process up to now")

Elapsed time: 147.96743202209473 seconds for process up to now


In [20]:
input("STOP! Run the Model Enhancements macro first. Then press Enter to continue")

STOP! Run the Model Enhancements macro first. Then press Enter to continue 


''

## Refreshing Golden Source ##

In [21]:
g_src = dst_folder+"\Golden PowerBI Source File_"+date_str+".xlsx"

gs_df = pd.read_excel(g_src, sheet_name="Golden - Spend Tracking", header=0)
#gs_df

model = dst_folder+"\Model Enhancements_"+date_str+".xlsm"

mod_df = pd.read_excel(model, sheet_name="Combined", header=0)
#mod_df

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:6: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:6: SyntaxWarning: invalid escape sequence '\M'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\490345161.py:1: SyntaxWarning: invalid escape sequence '\G'
  g_src = dst_folder+"\Golden PowerBI Source File_"+date_str+".xlsx"
C:\Users\stephalin\AppData\Local\Temp\ipykernel_31740\490345161.py:6: SyntaxWarning: invalid escape sequence '\M'
  model = dst_folder+"\Model Enhancements_"+date_str+".xlsm"


In [22]:
#pivot combined tab in TBM Model file by Org Symbol,  Oblig, Expenditure
#filter out CURRENT month, not month of dashboard
month_to_filter = today.strftime("%b").upper() + today.strftime("-%y")
filtered = mod_df[mod_df["Accounting Period"] != month_to_filter]
pivot = filtered.pivot_table(
    index="Org Symbol",
    values=["Obligations", "Expenditures", "Ceiling"],
    aggfunc="sum"
)
pivot

,Ceiling,Expenditures,Obligations
Org Symbol,,,
CEO,53306760.0,23241997.60,32878667.00
CSPO,48375020.0,16852427.50,40067184.64
DSO,205536508.0,61900105.27,98501212.23
OD,72479335.0,30488491.70,54124365.37


In [23]:
#throw them into cum oblig , cum exp on golden source file
this_year = prior_month_date.year
fy_str = f"FY{str(this_year)[-2:]}"  # "FY25"
monthtofill = (gs_df["FY"] == fy_str) & (gs_df["Month"] == month_str)

# Ensure the lengths of mask and pivot values match
mask = (gs_df["FY"] == fy_str) & (gs_df["Month"] == month_str)
rows_to_update = gs_df.loc[mask]

# Check if lengths match before assignment
if len(rows_to_update) == len(pivot):
    gs_df.loc[mask, "Cumulative Obl"] = pivot["Obligations"].values
    gs_df.loc[mask, "Cumulative Exp"] = pivot["Expenditures"].values
    gs_df.loc[mask, "Budget Ceiling"] = pivot["Ceiling"].values
else:
    print("Row count mismatch: cannot assign values safely.")

gs_df[mask]

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
228,CUSTOMER ENGAGEMENT OFFICE,July,10,53350313,53306760.0,NaN,NaN,32878667.00,23241997.60,4.901203e+07,CEO,FY25
229,CYBERSECURITY PROGRAM OFFICE,July,10,45536505,48375020.0,NaN,NaN,40067184.64,16852427.50,3.802060e+07,CSPO,FY25
230,DIGITAL SERVICES OFFICE,July,10,215002493,205536508.0,NaN,NaN,98501212.23,61900105.27,1.198878e+08,DSO,FY25
231,OFFICE OF THE DIRECTOR,July,10,51740257,72479335.0,NaN,NaN,54124365.37,30488491.70,5.748370e+07,OD,FY25


In [24]:
#Calculate Obligations and Expenditures (this month's Cum Obl or Exp - last month's)
months_order = ["October", "November", "December", "January", "February", "March", "April", "May", "June","July", "August", "September"]
gs_df["Month"] = pd.Categorical(gs_df["Month"], categories=months_order, ordered=True)

prev_mask = (gs_df["FY"] == fy_str) & (gs_df["Month"] == gs_df["Month"].cat.categories[gs_df["Month"].cat.categories.get_loc(month_str)-1])

gs_df.loc[mask, "Obligations"] = gs_df.loc[mask, "Cumulative Obl"].values - gs_df.loc[prev_mask, "Cumulative Obl"].values
gs_df.loc[mask, "Expenditures"] = gs_df.loc[mask, "Cumulative Exp"].values - gs_df.loc[prev_mask, "Cumulative Exp"].values

#gs_df[mask]

In [25]:
#ensure budget ceiling #s are consistent for remainder of year

months_order = ["October", "November", "December", "January", "February", "March", "April", "May", "June","July", "August", "September"]
gs_df["Month"] = pd.Categorical(gs_df["Month"], categories=months_order, ordered=True)

source_mask = (gs_df["Month"] == month_str) & (gs_df["FY"] == fy_str)
values = gs_df.loc[source_mask, "Budget Ceiling"].values

after_mask = (gs_df["Month"] >= month_str) & (gs_df["FY"] == fy_str)
n = after_mask.sum()
gs_df.loc[after_mask, "Budget Ceiling"] = np.tile(values, int(np.ceil(n/len(values))))[:n]
gs_df[after_mask]

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
228,CUSTOMER ENGAGEMENT OFFICE,July,10,53350313,53306760.0,-13070756.47,6585115.88,32878667.00,23241997.60,4.901203e+07,CEO,FY25
229,CYBERSECURITY PROGRAM OFFICE,July,10,45536505,48375020.0,7173193.41,1261596.48,40067184.64,16852427.50,3.802060e+07,CSPO,FY25
230,DIGITAL SERVICES OFFICE,July,10,215002493,205536508.0,2309364.92,6655788.10,98501212.23,61900105.27,1.198878e+08,DSO,FY25
231,OFFICE OF THE DIRECTOR,July,10,51740257,72479335.0,374908.70,6041722.72,54124365.37,30488491.70,5.748370e+07,OD,FY25
232,CUSTOMER ENGAGEMENT OFFICE,August,11,53350313,53306760.0,NaN,NaN,NaN,NaN,5.207464e+07,CEO,FY25
233,CYBERSECURITY PROGRAM OFFICE,August,11,45536505,48375020.0,NaN,NaN,NaN,NaN,4.314721e+07,CSPO,FY25
234,DIGITAL SERVICES OFFICE,August,11,215002493,205536508.0,NaN,NaN,NaN,NaN,1.435838e+08,DSO,FY25
235,OFFICE OF THE DIRECTOR,August,11,51740257,72479335.0,NaN,NaN,NaN,NaN,6.121795e+07,OD,FY25
236,CUSTOMER ENGAGEMENT OFFICE,September,12,53350313,53306760.0,NaN,NaN,NaN,NaN,5.513725e+07,CEO,FY25
237,CYBERSECURITY PROGRAM OFFICE,September,12,45536505,48375020.0,NaN,NaN,NaN,NaN,4.827381e+07,CSPO,FY25


In [26]:
GL_df = pd.read_excel(model, sheet_name="GL + Lookups", header=0)
#GL_df

In [27]:
# Fiscal quarters: 3 months each
quarters = [months_order[i:i+3] for i in range(0, 12, 3)]

#month_str = 'October'
month_idx = months_order.index(month_str)

# Find current quarter
current_qtr_idx = month_idx // 3

#Calculate Upcoming quarters (excluding current)
upcoming_quarters = len(quarters) - (current_qtr_idx + 1)
upcoming_quarters

0

In [28]:
#pivot by OCIO office, total proj oblig (only for Q4)
#For Q1-3, add UFMS commitments [QTR X] + Manual commitments [QTR X] + Other Projections [QTR X] + Payroll Proj [QTR X] + UFMS Obligations [QTR X] + UFMS Oblig
#Do this cumulatively

quarters = [
    ["UFMS Commitments Qtr 1", "Manual Commitments Qtr 1", "Other Projections Qtr 1", "Payroll Projections Qtr 1", "UFMS Obligations Qtr 1", "UFMS Obligations"],
    ["UFMS Commitments Qtr 2", "Manual Commitments Qtr 2", "Other Projections Qtr 2", "Payroll Projections Qtr 2", "UFMS Obligations Qtr 2", "UFMS Obligations"],
    ["UFMS Commitments Qtr 3", "Manual Commitments Qtr 3", "Other Projections Qtr 3", "Payroll Projections Qtr 3", "UFMS Obligations Qtr 3", "UFMS Obligations"]
]

# Cumulative sums for Qtr 1, Qtr 2, Qtr 3
for i in range(3):
    cols = [col for q in quarters[:i+1] for col in q if col in GL_df.columns]
    GL_df[f"Qtr_{i+1}"] = GL_df[cols].sum(axis=1)

# Qtr 4 is just "Total Projected Obligations"
GL_df["Qtr_4"] = GL_df["Total Projected Obligations"]

Val = [f"Qtr_{i+1}" for i in range(4)]

pivot_GL = GL_df.pivot_table(
    index="OCIO Office",
    values=Val,
    aggfunc="sum"
)

pivot_GL


,Qtr_1,Qtr_2,Qtr_3,Qtr_4
OCIO Office,,,,
CEO,4.540621e+07,8.960501e+07,1.497557e+08,4.854572e+07
CSPO,4.556225e+07,1.004974e+08,1.624336e+08,4.870870e+07
DSO,1.101689e+08,2.676307e+08,4.029741e+08,1.779420e+08
OD,5.918190e+07,1.561609e+08,2.195335e+08,6.437825e+07


In [29]:
#delete oblig projections for clean slate
gs_df['Obligation Projection'] = np.nan

#fill in Oblig Projection based on GL Lookup pivot #s. quarter months still upcoming will be imputed
if upcoming_quarters > 0:
    quarter_months = {1: "December", 2: "March", 3: "June", 4: "September"}
    for q in range(1, upcoming_quarters + 1):
        month = quarter_months[q]
        mask = (gs_df["Month"] == month) & (gs_df["FY"] == fy_str)
        gs_df.loc[mask, "Obligation Projection"] = gs_df.loc[mask, "Org Symbol"].map(pivot_GL[f"Qtr_{q}"])
elif upcoming_quarters == 0:
    mask = (gs_df["Month"] == "September") & (gs_df["FY"] == fy_str)
    gs_df.loc[mask, "Obligation Projection"] = gs_df.loc[mask, "Org Symbol"].map(pivot_GL["Qtr_4"])


gs_df[after_mask]

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
228,CUSTOMER ENGAGEMENT OFFICE,July,10,53350313,53306760.0,-13070756.47,6585115.88,32878667.00,23241997.60,NaN,CEO,FY25
229,CYBERSECURITY PROGRAM OFFICE,July,10,45536505,48375020.0,7173193.41,1261596.48,40067184.64,16852427.50,NaN,CSPO,FY25
230,DIGITAL SERVICES OFFICE,July,10,215002493,205536508.0,2309364.92,6655788.10,98501212.23,61900105.27,NaN,DSO,FY25
231,OFFICE OF THE DIRECTOR,July,10,51740257,72479335.0,374908.70,6041722.72,54124365.37,30488491.70,NaN,OD,FY25
232,CUSTOMER ENGAGEMENT OFFICE,August,11,53350313,53306760.0,NaN,NaN,NaN,NaN,NaN,CEO,FY25
233,CYBERSECURITY PROGRAM OFFICE,August,11,45536505,48375020.0,NaN,NaN,NaN,NaN,NaN,CSPO,FY25
234,DIGITAL SERVICES OFFICE,August,11,215002493,205536508.0,NaN,NaN,NaN,NaN,NaN,DSO,FY25
235,OFFICE OF THE DIRECTOR,August,11,51740257,72479335.0,NaN,NaN,NaN,NaN,NaN,OD,FY25
236,CUSTOMER ENGAGEMENT OFFICE,September,12,53350313,53306760.0,NaN,NaN,NaN,NaN,4.854572e+07,CEO,FY25
237,CYBERSECURITY PROGRAM OFFICE,September,12,45536505,48375020.0,NaN,NaN,NaN,NaN,4.870870e+07,CSPO,FY25


In [30]:

#Cum Obl for this month is Obl Prjection
mask = gs_df["Month"] == month_str
gs_df.loc[mask, "Obligation Projection"] = gs_df.loc[mask, "Cumulative Obl"]

#gs_df[after_mask]

In [31]:
# Sort the DataFrame
gs_df.sort_values(by=["FY", "Office", "Month Order"], inplace=True)

# Interpolate missing values in 'Obligation Projection'
gs_df['Obligation Projection'] = gs_df.groupby(['FY', 'Office'])['Obligation Projection'].transform(lambda group: group.interpolate())
#re-sort
gs_df.sort_values(by=["FY", "Month Order","Office"], inplace=True)

#gs_df[after_mask]

In [32]:
with pd.ExcelWriter(g_src, mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    gs_df.to_excel(writer, sheet_name='Golden - Spend Tracking', index=False)

In [33]:
end = time.time()
print(f"Elapsed time: {(end - start_master)/60} minutes TOTAL")

Elapsed time: 3.4429689407348634 minutes TOTAL
